In [ ]:
# Strategies
# 1 - Train model from scratch
# 2 - Feature extraction using pre-trained models
# 3 - Fine tunning pre-trained model

In [ ]:
import os

train_dir = "Dataset/train/"
test_dir = "Dataset/test/"
validation_dir = "Dataset/validation/"

train_rice_dir = os.path.join(train_dir, 'rice')
validation_rice_dir = os.path.join(validation_dir, 'rice')
test_rice_dir = os.path.join(test_dir, 'rice')


In [ ]:
from keras import layers
from keras import models
from keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

# Added a dropout layer
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
                optimizer=optimizers.RMSprop(lr=1e-4), 
                metrics=['acc'])

model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Added data augmentation: only the train dataset is augmented!
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180, 
    width_shift_range=0.25,
    height_shift_range=0.25, 
    shear_range=0.25,
    zoom_range=0.1, 
    horizontal_flip=True, 
    fill_mode='nearest')


# Pre-process the dataset: rescale to 1 channel and
target_size=(150, 150)
batch_size = 3

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( 
        train_dir, 
        target_size=target_size, 
        batch_size=batch_size, 
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir, 
    target_size = target_size, 
    batch_size=batch_size, 
    class_mode='binary')

In [ ]:
# Train the model
history = model.fit_generator(train_generator, 
    steps_per_epoch=20, 
    epochs=40, 
    validation_data=validation_generator,  
    validation_steps=50)

model.save('rice_tuna_tomato_2.h5')

In [ ]:
# Plot the results on this training
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend(), plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(), plt.show()

In [ ]:
# Data augmentation
from keras.preprocessing import image

img_path = "Dataset/train/rice/1413412efd - Cópia (2).jpg"
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

datagen = ImageDataGenerator(
    rotation_range=180, width_shift_range=0.25,
    height_shift_range=0.25, shear_range=0.25,
    zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break
plt.show()  



One might train a network on ImageNet (where classes are mostly animals and everyday objects) and then repurpose this trained network for something as remote as identifying furniture items in images. Such portability of learned features across different problems is a key advantage of deep learning compared to many older, shallow-learning approaches, and it makes deep learning very effective for small-data problems.

Dataset --> ImageNet

Model / architecture --> VGG16


There are two ways to use a pretrained network: feature extraction and fine-tuning.